<span style="float:right">
<a href="http://moldesign.bionano.autodesk.com/" target="_blank" title="About">About</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://forum.bionano.autodesk.com/c/Molecular-Design-Toolkit" target="_blank" title="Forum">Forum</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/autodesk/molecular-design-toolkit/issues" target="_blank" title="Issues">Issues</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="http://bionano.autodesk.com/MolecularDesignToolkit/explore.html" target="_blank" title="Tutorials">Tutorials</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<a href="http://autodesk.github.io/molecular-design-toolkit/" target="_blank" title="Documentation">Documentation</a></span>
</span>
![Molecular Design Toolkit](img/top.png)
<br>
<center><h1>Example 2: Orbitals, Wavefunctions, and Potential Energy Surfaces </h2> </center>

---

This notebook shows a few different ways that Molecular Design Toolkit can be used to run and visualize electronic structure calculations.

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib.pylab import *

import moldesign as mdt
from moldesign import units as u

<h1>Contents</h1>

---

   - [I. Molecular hydrogen](#I.-Molecular-hydrogen)
     - [A. Build the molecule](#A.-Build-the-molecule)
     - [B. Run a hartree-fock calculation](#B.-Run-a-hartree-fock-calculation)
     - [C. Visualize the orbitals](#C.-Visualize-the-orbitals)
     - [D. Minimize the energy](#D.-Minimize-the-energy)
     - [E. Scan the potential energy surface](#E.-Scan-the-potential-energy-surface)
   - [II. Butadiene](#II.-Butadiene)
     - [A. Build the molecule and calculate its energy](#A.-Build-the-molecule-and-calculate-its-energy)
     - [B. Select a bond](#B.-Select-a-bond)
     - [C. Calculate the initial dihedral angle](#C.-Calculate-the-initial-dihedral-angle)
     - [D. Scan over the dihedral angle](#D.-Scan-over-the-dihedral-angle)
     - [E. Minimize the new form](#E.-Minimize-the-new-form)
   - [III. Advanced topics](#III.-Advanced-topics)
     - [A. Symmetry](#A.-Symmetry)
     - [B. Examining the wavefunction](#B.-Examining-the-wavefunction)

## I. Molecular hydrogen

### A. Build the molecule
This cell builds H<sub>2</sub> by creating the two atoms, and explicitly setting their positions.

**Try editing this cell to**:
 * Create HeH<sup>+</sup>
 * Create H<sub>3</sub><sup>+</sup>
 * Change the atoms' initial positions

In [ ]:
atom1 = mdt.Atom('H')
atom2 = mdt.Atom('H')
atom1.bond_to(atom2,1)
atom2.x = 2.0 * u.angstrom

h2 = mdt.Molecule([atom1,atom2], name='H2', charge=0)
h2.draw(height=300)

### B. Run a hartree-fock calculation
The next cell adds the RHF energy model to our molecule, then triggers a calculation.

**Try editing this cell to**:
 * Change the atomic basis
 * Get a list of other available energy models (type `mdt.models.` and then hit the `[tab]` key)

In [ ]:
h2.set_energy_model(mdt.models.RHF, basis='3-21g')
h2.calculate()

print 'Calculated properties:', h2.properties.keys()
print 'Potential energy:', h2.potential_energy

### C. Visualize the orbitals
After running the calculation, we have enough information to visualize the molecular orbitals.

In [ ]:
h2.draw_orbitals()

### D. Minimize the energy
Here, we'll run a quick energy minimization then visualize how the hydrogen nuclei AND the atomic wavefunctions changed.

In [ ]:
minimization = h2.minimize(frame_interval=1, nsteps=10)
minimization.draw_orbitals()

In [ ]:
minimization

### E. Scan the potential energy surface
This cell calculates the potential energy at a series of interatomic separations, from 0.3 to 7.0 Å; the results are collections in a `Trajectory` object, named `scan` here.

In [ ]:
distances = np.arange(0.3,7.0,0.5)*u.angstrom
scan = mdt.Trajectory(h2)
h2.atoms[0].x = 0.0*u.angstrom
for r in distances:
    h2.atoms[1].x = r
    h2.calc_potential_energy()
    scan.new_frame(annotation='Separation: %s'%r)

scan.draw_orbitals()

The results can also be plotted directly in `matplotlib`:

In [ ]:
plot(distances, scan.potential_energy)
grid(); xlabel('separation / ang'); ylabel('energy / eV')

figure()
plot(distances, scan.atoms[0].mulliken, label='atom 1 partial charge')
plot(distances, scan.atoms[1].mulliken, label='atom 2 partial charge')
grid(); xlabel('separation / ang'); ylabel('partial charge'); legend()

## II. Butadiene
This example deals with <a href="https://en.wikipedia.org/wiki/1,3-Butadiene">butadiene</a>, a slightly larger molecule. Here, we'll scan its potential energy surface over its central dihedral angle.

### A. Build the molecule and calculate its energy
The following code cell uses a <a href="https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system">SMILES string</a> to create the molecule.

In [ ]:
mol = mdt.from_smiles('C=CC=C')
mol.set_energy_model(mdt.models.RHF(basis='sto-3g'))
result = mol.calculate(['potential_energy','orbitals'], wait=True)

print 'Potential energy:',result.potential_energy.to(u.kcalpermol)
mol.draw_orbitals()

### B. Select a bond
Next, we pull up an interactive widget to select a specific bond.

In [ ]:
selector = mdt.widgets.BondSelector(mol)
selector

Click on any bond you like in the above widget. We can programatically access that selection using `selector.selected_bonds`:

In [ ]:
selector.selected_bonds

### C. Calculate the initial dihedral angle
Next, we'll gather the four atoms involved in the central dihedral and calculate the current angle.

In [ ]:
bond = selector.selected_bonds[0]
a1 = bond.a1
a2 = bond.a2
a0 = [atom for atom in a1.bond_graph 
      if atom.atnum==6 and atom is not a2][0]
a3 = [atom for atom in a2.bond_graph 
      if atom.atnum==6 and atom is not a1][0]

print 'Dihedral:', mdt.dihedral(a0, a1, a2, a3).to(u.degrees)

### D. Scan over the dihedral angle

This code loops over values of the dihedral angle from 180º to 360º, calcualting the wavefunction along the way.

In [ ]:
traj = mdt.Trajectory(mol)
traj.new_frame()
for angle in np.arange(180, 360.0, 15.0)*u.degrees:
    mdt.set_dihedral(a0, a1, a2, a3, angle)
    mol.calculate()
    traj.new_frame(annotation='dihedral:%s' % angle) 

In [ ]:
traj.draw_orbitals()

### E. Minimize the new form

In the new *cis* form, we search for a new energy minimum:

In [ ]:
mintraj = mol.minimize(frame_interval=1)

In [ ]:
plot(mintraj.potential_energy); ylabel('energy / eV'); grid()
mintraj.draw_orbitals()

## III. Advanced topics

### A. Symmetry
You can use buckyball's symmetrization tools to build symmetric molecules, which can often lead to huge improvements in computational efficiency.

First, let's look at the simplest possible organic molecule, methane. Its simplicity actually leads to some very complex symmetry.

In [ ]:
methane = mdt.from_name('methane')
mdt.widgets.Symmetrizer(methane)

The list of items on the right is the list of individual symmetry elements - for methane, there are a lot. Click on one to see it illustrated in 3D. Larger molecules will generally have far less symmetry.

Let's take a look at ethane. We'll add some noise to the molecule's coordinates to destroy exact symmetry.

In [ ]:
ethane = mdt.from_smiles('CC')
for atom in ethane.atoms: atom.position += 0.1 * u.angstrom * np.random.random(3)
mdt.widgets.Symmetrizer(ethane)

You can use the "symmetrize" button to impose individual symmetry elements exactly.

### B. Examining the wavefunction
You can dive deeply into the quantum wavefunction results by examining the `h2.electronic_state` object. For instance, we can verify that the MO basis diagonalizes the Fock matrix:

In [ ]:
fock_ao = h2.wfn.fock_ao
mos_in_ao = h2.wfn.molecular_orbitals

matshow( h2.wfn.orbitals.canonical.fock )
title('Fock matrix in canonical MO basis'); cb = colorbar()
cb.set_label('energy / eV')

matshow( h2.wfn.orbitals.atomic.fock )
title('Fock matrix in atomic orbital basis'); cb = colorbar()
cb.set_label('energy / eV')